In [17]:
import json
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

In [18]:

with open('document_with_ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [19]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [21]:
es_client = Elasticsearch("http://localhost:9200")

index_settings = {
    "settings": {
        "number_of_shards" : 1,
        "number_of_replicas" : 0
    },
    "mappings" : {
        "properties" : {
            "text" : {"type" : "text"},
            "section" : {"type" : "text"},
            "question" : {"type" : "text"},
            "course" : {"type" : "text"},
            "id" : {"type" : "keyword"},
        }
    }
}

index_name = "course_questions"
es_client.indices.delete(index = index_name, ignore_unavailable=True)
es_client.indices.create(index = index_name, body = index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course_questions'})

In [22]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 338.79it/s]


In [27]:
def elastic_search(query,course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index= index_name, body= search_query)     
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        
    print(len(result_docs))
    return result_docs

In [30]:
elastic_search(
    query="Can I still join?",
    course="data-engineering-zoomcamp"
)

[]